In [2]:
import pandas as pd
import tweepy
from textblob import TextBlob


In [3]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [4]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

polarities = {1:'Very Negative',2:'Negative', 3:'Neutral', 4:'Positive', 5:'Very Positive'}

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
#     tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
#                                reduce_len=True)
#     tweet_tokens = tokenizer.tokenize(tweet)

#     tweets_clean = []
#     for word in tweet_tokens:
#         if (word not in stopwords_english and  # remove stopwords
#             word not in string.punctuation):  # remove punctuation
#             # tweets_clean.append(word)
#             stem_word = stemmer.stem(word)  # stemming word
#             tweets_clean.append(stem_word)

    return tweet


def test_lookup(func):
    freqs = {('sad', 0): 4,
             ('happy', 1): 12,
             ('oppressed', 0): 7}
    word = 'happy'
    label = 1
    if func(freqs, word, label) == 12:
        return 'SUCCESS!!'
    return 'Failed Sanity Check!'


def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n



def normalize_sentiment(val):
    
    senMin = -1
    senMax = 1
    polMin = 1
    polMax =5

    newval = ((val -senMin)/(senMax-senMin))*(polMax-polMin)+polMin
        
    return round(newval)

In [2]:
#Aim is to get tweets based on hashtags which has covid-19 apps, security, covid19.
#Data Extraction first
#Reference : https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

In [5]:
access_token_key='1097367016456011776-bKEHl6bEgWc7gnd3cIqu3KlYsWgoi4'
access_token_secret='k8CVegeF8hzHJJQDTWEqQ1rzUVYn228TK5eMaO8Pan2M9'
consumer_key='CdNDp6wqPDmgUAQqCkikjNmTG'
consumer_secret='jTSjfniCLUEdmLplHGEniFVz0L2Ha7yfrWhxUvOGBWuN51OOyQ'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)

auth.set_access_token(access_token_key,access_token_secret)

In [6]:
twtr = tweepy.API(auth,wait_on_rate_limit=True)

In [7]:
covid_keys = '#covid19apps OR #covidtracker OR #covidappsecurity'

In [20]:
counter = 0
new_entry=[]
for page in tweepy.Cursor(twtr.search,q=covid_keys,count=200,include_rts=False).pages():
    for status in page:
        
        status = status._json
        #print(status['text'])
        twt=process_tweet(status['text'])
        opinion = polarities[normalize_sentiment(TextBlob(twt).sentiment.polarity)]
        timestamp = status['created_at']
        counter+=1
        
        new_entry.append([counter,twt,timestamp,opinion])
        
        
    
 

In [ ]:
#Five categories: 
#{1 to be strongly negative, 2 to be negative, 3 to be neutral, 4 to be positive, and 5 to be strongly positive}

In [36]:
# 1=5
# -1=1


#normalization formula from:https://stats.stackexchange.com/questions/281162/scale-a-number-between-a-range

    #   or
#   newvalue= (max'-min')/(max-min)*(value-min)+min'.

In [24]:
df = pd.DataFrame(new_entry, columns=['id','Tweet','time','Opinion'])

In [33]:
import numpy as np
np.unique(df['Opinion'],return_counts=True)

(array(['Negative', 'Neutral', 'Positive', 'Very Negative',
        'Very Positive'], dtype=object),
 array([  49, 2739, 2506,   16,   62], dtype=int64))

In [ ]:
df.to_csv('')